# Data Cleaning

This notebook cleans the raw supply chain datasets by fixing data types,
removing invalid records, and preparing consistent data for further
analysis and feature engineering.


In [71]:
import pandas as pd
import numpy as np


In [72]:
orders = pd.read_csv("../data/raw/orders_and_shipments.csv")
inventory = pd.read_csv("../data/raw/inventory.csv")
fulfillment = pd.read_csv("../data/raw/fulfillment.csv")

orders.columns = orders.columns.str.strip()
inventory.columns = inventory.columns.str.strip()
fulfillment.columns = fulfillment.columns.str.strip()


In [73]:
orders = orders.drop(
    columns=["Order Item ID", "Order Time"],
    errors="ignore"
)


In [74]:
orders.columns


Index(['Order ID', 'Order YearMonth', 'Order Year', 'Order Month', 'Order Day',
       'Order Quantity', 'Product Department', 'Product Category',
       'Product Name', 'Customer ID', 'Customer Market', 'Customer Region',
       'Customer Country', 'Warehouse Country', 'Shipment Year',
       'Shipment Month', 'Shipment Day', 'Shipment Mode',
       'Shipment Days - Scheduled', 'Gross Sales', 'Discount %', 'Profit'],
      dtype='object')

In [75]:
orders["order_date"] = pd.to_datetime(
    {
        "year": orders["Order Year"],
        "month": orders["Order Month"],
        "day": orders["Order Day"]
    }
)


In [76]:
orders["shipment_date"] = pd.to_datetime(
    {
        "year": orders["Shipment Year"],
        "month": orders["Shipment Month"],
        "day": orders["Shipment Day"]
    }
)


In [77]:
orders[["order_date", "shipment_date"]].head()


,order_date,shipment_date
0,2015-02-21,2015-02-27
1,2015-03-02,2015-03-06
2,2015-04-18,2015-04-20
3,2015-06-10,2015-06-12
4,2015-06-10,2015-06-12


In [78]:
orders["shipping_time"] = (
    orders["shipment_date"] - orders["order_date"]
).dt.days


In [79]:
orders = orders[
    (orders["shipping_time"] >= 0) &
    (orders["shipping_time"] <= 28)
]


In [80]:
orders["Customer Country"] = (
    orders["Customer Country"]
    .astype(str)
    .str.encode("latin1", errors="ignore")
    .str.decode("utf-8", errors="ignore")
)

In [81]:
orders = orders.dropna()

In [82]:
orders = orders.drop_duplicates()
inventory = inventory.drop_duplicates()
fulfillment = fulfillment.drop_duplicates()


In [83]:
orders["Discount %"] = (
    orders["Discount %"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.strip()
)
orders["Discount %"] = pd.to_numeric(
    orders["Discount %"],
    errors="coerce"
)


In [84]:
orders.to_csv("../data/processed/orders_clean.csv", index=False)
inventory.to_csv("../data/processed/inventory_clean.csv", index=False)
fulfillment.to_csv("../data/processed/fulfillment_clean.csv", index=False)
